# Грузим библиотеки

In [ ]:
from flask import Flask
from flask import request
import requests
from flask import make_response
import os
import json
from pandas import DataFrame
import traceback


from methods import access
from methods.emoji import emoji
from methods import telegram_bot_methods
from methods import google_maps_methods
from methods import psql_methods
from methods import reply



import app



if __name__ == "__main__":

    
    app.application.run()



In [ ]:
X = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}
X['message']['text'] = '/start'
X['message']['chat']['last_name']

In [ ]:
#тест отправки графика
import psycopg2
from pandas import DataFrame
import pandas as pd
from datetime import datetime
from datetime import timedelta

import numpy as np

#текущая метка времени
def now_str():
    now = datetime.now()
    now_str = str(now.year)+str(now.month if now.month >= 10 else  '0'+str(now.month))+str(now.day if now.day >= 10 else  '0'+str(now.day)) +' '+str(now.hour if now.hour >= 10 else  '0'+str(now.hour)) + str(now.minute if now.minute >= 10 else  '0'+str(now.minute)) + str(now.second if now.second >= 10 else  '0'+str(now.second))
    return now_str


dbname = 'dbr3jigs1op5oo'
port = '5432'
user = 'muwrkppfuyldmk'
host = 'ec2-54-227-252-202.compute-1.amazonaws.com'
password = '4c4eabfcaf92f7289ccfc1a314d04a3c3806db72b1bf12fd5f0f40c410b14355'

PSQL_heroku_keys = {'dbname' : dbname
                    , 'port' : port
                    , 'user' : user
                    , 'host' : host
                    , 'password' : password
                    }


user_id = 13
command = 'На текущий месяц'
command = ''


conn = psycopg2.connect("dbname='%(dbname)s' port='%(port)s' user='%(user)s' host='%(host)s' password='%(password)s'" % PSQL_heroku_keys)



#формат ответа
response = {'status' : 200
            ,'report' : 'delete_transaction_plan'
            ,'system_message' : 'No report'
            ,'text' : None
            ,'reply_markup' : None
            }

# создаем запрос
cur = conn.cursor()

if command == 'На сегодня':
    cur.execute("select * from public.tasks where  user_id = %(user_id)s and date_task >= date_trunc('day', now()) and date_task < date_trunc('day', now()) +  interval '1 day'" % {  'user_id' : user_id} )
elif command == 'На текущую неделю':
    cur.execute("select * from public.tasks where  user_id = %(user_id)s and date_task >= date_trunc('week', now()) and date_task < date_trunc('week', now()) + interval '7 day'" % {  'user_id' : user_id} )
elif command == 'На текущий месяц':
    cur.execute("select * from public.tasks where  user_id = %(user_id)s and date_task >= date_trunc('month', now()) and date_task < date_trunc('month', now()) + interval '1 month'" % {  'user_id' : user_id} )
else:
    cur.execute("select * from public.tasks where  user_id = %(user_id)s " % {  'user_id' : user_id} )


    
df_plan_tasks = DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description]).sort_values(by='date_task' , ascending=True)

cur.close() 

if df_plan_tasks.shape[0] == 0:
    response['text'] = command + ' дел нет.'
else:
    text = command + ' следующие дела:\n\n'
    num = 1
    for row in df_plan_tasks.iterrows():
        text += str(num) + ': ' + row[1]['task'] + ' ' + row[1]['date_task'].strftime('%d.%m.%Y') + '\n'
        num += 1
        
        
    response['text'] = text
    


print ('ok')


df_plan_tasks

In [ ]:
text = command + ' следующие дела:\n\n'
num = 1
for row in df_plan_tasks.iterrows():
    text += str(num) + ': ' + row[1]['task'] + ' ' + row[1]['date_task'].strftime('%d.%m.%Y') + '\n'

    num += 1


In [ ]:
print (text)

In [ ]:
df_plan_tasks['date_task'].dt.strftime('%d.%m.%Y')

# make some graphics

In [ ]:


df_plan_tasks['YearMonth'] = pd.to_datetime(df_plan_tasks['date_task']).map(lambda dt: dt.replace(day=1))

res = df_plan_tasks.groupby(['YearMonth'])['id'].count()
res_6 = res.sort_index(ascending = False).head(6).sort_index(ascending = True)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

month_dict= {1:'Январь',
            2:'Февраль',
            3:'Март',
            4:'Апрель',
            5:'Май',
            6:'Июнь',
            7:'Июль',
            8:'Август',
            9:'Сентябрь',
            10:'Октябрь',
            11:'Ноябрь',
            12:'Декабрь'}


res_6.index = [month_dict[x.month] for x in list(res_6.index)]

res_6.plot(kind='bar' )



In [ ]:
#номер недели
datetime.date(df_plan_tasks['date_task'][1]).isocalendar()[1]

#сделать - по неделям разбить месяц и как в течение месяца делались дела, сравнивать с средним значением по месяцам
#в конце месяца отчетик по тому, как нагружаешь себя и как выполняешь задачи, как прокачиваешься

In [ ]:
#отчет по делам по итогам месяца
#соираем информацию о проделанных делах
#хотим показать - по загрузке по неделям как дела ставили - сравнение с среднестатистической загрузкой нашей понедельной.
#рекомендации и похвалы по итогу сравнения. 
#Смотреть самые популярные темы по итогу месяца какие дела себе ставил по итогу сематического анализа
from dateutil.relativedelta import relativedelta
now_date = datetime.now()
next_date = datetime.now()+ relativedelta(months=1)
prev_date = datetime.now()- relativedelta(months=1)


now_year = now_date.year
now_month = now_date.month

prev_year = prev_date.year
prev_month = prev_date.month

datetime(now_year, now_month , 1).isocalendar()[1]



In [ ]:


df_prev_month = df_plan_tasks[(df_plan_tasks['date_task'] >= datetime(prev_year, prev_month , 1))
             & (df_plan_tasks['date_task'] < datetime(now_year, now_month , 1))][:]

df_prev_month['n_week'] = [int(x.day / 7) + 1 for x in df_prev_month['date_task']]

res = df_prev_month.groupby(['n_week'])['id'].count()

In [ ]:

cnt_missions = res.values
name_bars = list(res.index)
y_pos = np.arange(len(name_bars))


plt.bar(y_pos, cnt_missions, color = (0.2, 0.9, 0.7, 0.8) ,  edgecolor='gray')



plt.xlabel('неделя')
plt.ylabel('выполнено дел')


plt.ylim(0,50)


plt.xticks(y_pos, name_bars)

for a,b in zip(y_pos, cnt_missions): 
    plt.text(a - 0.1, b + 1, str(b) , fontsize=11 , color = 'darkgreen')


plt.savefig('to1.png' ,facecolor='w', edgecolor='w')

plt.show()



In [ ]:
from PIL import Image, ImageDraw
image = Image.open('to1.png')
image.show()

#для отправки в телеграме 
x = open('to1.png', 'rb')


bot.send_photo(chat_id=chat_id, photo=open('tests/test.png', 'rb'))


In [ ]:
import os
os.remove("to2.png")


In [ ]:
plt.savefig('to2.png' ,facecolor='w', edgecolor='w')
x = open('to2.png', 'rb')
x.close()
#os.remove("to2.png")
